In [275]:
import pandas as pd
df_en = pd.read_csv(r'C:\Users\XE\Downloads\1mcorpus\corpus.en_ru.1m.en', encoding="utf-8", names=["seq_en"], sep='\r\n')
df_ru = pd.read_csv(r'C:\Users\XE\Downloads\1mcorpus\corpus.en_ru.1m.ru', encoding="utf-8", names=["seq_ru"], sep='\r\n')

C:\Users\XE\AppData\Local\Temp\ipykernel_3788\3478127182.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_en = pd.read_csv(r'C:\Users\XE\Downloads\1mcorpus\corpus.en_ru.1m.en', encoding="utf-8", names=["seq_en"], sep='\r\n')
C:\Users\XE\AppData\Local\Temp\ipykernel_3788\3478127182.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_ru = pd.read_csv(r'C:\Users\XE\Downloads\1mcorpus\corpus.en_ru.1m.ru', encoding="utf-8", names=["seq_ru"], sep='\r\n')


In [276]:
df_en = df_en.sample(100000, random_state=0)
df_ru = df_ru.sample(100000, random_state=0)

In [277]:
df_en['seq_en'].to_csv("../datasets/NMT_en")
df_ru['seq_ru'].to_csv("../datasets/NMT_ru")

In [278]:
df = df_en.join(df_ru)

In [279]:
df.to_csv("../datasets/NMT_en_ru")

In [280]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from torch.utils.data import TensorDataset, DataLoader
import re
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
from razdel import tokenize
from nltk.tokenize import word_tokenize
from navec import Navec

In [281]:
import torchtext
import torchdata
from torchtext import transforms
from torchdata import datapipes

In [282]:
data_pipe = datapipes.iter.IterableWrapper(["../datasets/NMT_en_ru"])
data_pipe_ru = datapipes.iter.IterableWrapper(["../datasets/NMT_ru"])
data_pipe_en = datapipes.iter.IterableWrapper(["../datasets/NMT_en"])

In [283]:
data_pipe = datapipes.iter.FileOpener(data_pipe, mode='r', encoding='utf-8')
data_pipe_ru = datapipes.iter.FileOpener(data_pipe_ru, mode='r', encoding='utf-8')
data_pipe_en = datapipes.iter.FileOpener(data_pipe_en, mode='r', encoding='utf-8')

In [284]:
data_pipe= data_pipe.parse_csv(skip_lines=0, as_tuple=True)
data_pipe_ru = data_pipe_ru.parse_csv(skip_lines=0, as_tuple=True)
data_pipe_en = data_pipe_en.parse_csv(skip_lines=0, as_tuple=True)

In [285]:
def tokenize_en(data: str) -> list[str]:
  data1 = re.sub("([.!?])", " ", str(data[1:]))
  data2 = re.sub("[^\s^\w^']+", " ", data1)
  data3 = word_tokenize(data2)
  return data3

In [286]:
def tokenize_ru(data: str) -> list[str]:
  data1 = re.sub("([.!?])", " ", str(data[1:]))
  data2 = re.sub("[^\s^\w^']+", " ", data1)
  data3 = tokenize(data2)
  data4 = list(map(lambda x: x.text , data3))
  return data4

In [287]:
def yield_tokens_en(data_iter: datapipes.iter.IterDataPipe):
  # итерируемся по набору данных
  for example in data_iter:
    yield tokenize_en(example)

In [288]:
def yield_tokens_ru(data_iter: datapipes.iter.IterDataPipe):
  # итерируемся по набору данных
  for example in data_iter:
    yield tokenize_ru(example)

In [289]:
from torchtext.vocab import build_vocab_from_iterator

In [290]:
vocab_en = build_vocab_from_iterator(
    yield_tokens_en(data_pipe_en),
    min_freq=1,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
vocab_en.set_default_index(vocab_en['<unk>'])

In [291]:
vocab_ru = build_vocab_from_iterator(
    yield_tokens_ru(data_pipe_ru),
    min_freq=1,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
vocab_ru.set_default_index(vocab_ru['<unk>'])

In [292]:
from torchtext import transforms

In [293]:
def vocab_transform(vocab: torchtext.vocab.Vocab) -> transforms.Sequential:
    text_tranform = transforms.Sequential(
        transforms.VocabTransform(vocab=vocab),
        transforms.AddToken(vocab['<sos>'], begin=True),
        transforms.AddToken(vocab['<eos>'], begin=False)
    )
    return text_tranform

In [294]:
def apply_vocab_transform(sample):
    return (
        vocab_transform(vocab_en)(tokenize_en(sample[1])),
        vocab_transform(vocab_ru)(tokenize_ru(sample[2]))
    )

In [295]:
data_pipe = data_pipe.map(apply_vocab_transform)

In [296]:
i=0
for sample in data_pipe:
    if i == 1660:    
        print(sample)
        break
    i+=1

([1, 3, 47, 1745, 46, 222, 5594, 2013, 8, 18, 14699, 254, 131, 1543, 12, 4, 205, 4474, 20772, 659, 10060, 2], [1, 3, 53, 31, 134, 7468, 2071, 24, 8812, 17056, 182925, 22, 1222, 6416, 119, 46027, 3030, 29, 81, 176964, 7175, 2])


In [297]:
data_pipe_new = []
for sample in data_pipe:
    data_pipe_new.append((torch.tensor(sample[0]), torch.tensor(sample[1])))

In [298]:
from torch.nn.utils.rnn import pad_sequence
en_text_padded = pad_sequence([tupl[0] for tupl in data_pipe_new], True)
ru_text_padded = pad_sequence([tupl[1] for tupl in data_pipe_new], True)

In [311]:
MAX_LENGTH_RU = 88
MAX_LENGTH_EN = 86

In [299]:
X_train, X_test, y_train, y_test = train_test_split(en_text_padded, ru_text_padded, test_size=0.001)

In [300]:
ds = TensorDataset(X_train, torch.tensor(y_train).type(torch.int64))
dl = DataLoader(ds, batch_size=32, shuffle=True)

C:\Users\XE\AppData\Local\Temp\ipykernel_3788\3442895299.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ds = TensorDataset(X_train, torch.tensor(y_train).type(torch.int64))


In [316]:
device = torch.device ( "cuda:0" if torch.cuda.is_available() else "cpu" )
print ( device )

cpu


In [331]:
class EncoderNMT(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, dropout_p=0.1):
        super(EncoderNMT, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [333]:
class DecoderNMT(nn.Module):
    def __init__(self, embed_size, hidden_size, output_size):
        super(DecoderNMT, self).__init__()
        self.embedding = nn.Embedding(output_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(vocab_ru['SOS'])
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH_RU):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        #decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = self.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [334]:
class NMT_model(nn.Module):
    def __init__(self):
        super(NMT_model, self).__init__()
        self.encoder = EncoderNMT(len(vocab_en), 128, 128)
        self.decoder = DecoderNMT(128, 128, len(vocab_ru))
    
    def forward(self, input, target):
        encoder_outputs, encoder_hidden = self.encoder(input)
        decoder_outputs, _, _ = self.decoder(encoder_outputs, encoder_hidden, target)
        return decoder_outputs

In [335]:
loss = nn.CrossEntropyLoss()
model = NMT_model()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [336]:
epochs = 10
for epoch in range(epochs):
    batch = 0
    for x_b, y_b in dl:
        x_b = x_b.to(device)
        y_b = y_b.to(device)
        optimizer.zero_grad()
        batch+=1
        decoder_outputs = model(x_b, y_b)

        loss_value = loss(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            y_b.view(-1))
        loss_value.backward()
        
        optimizer.step()
        
        #sys.stderr.write(f'Батч {batch + 1}/{len(dl)}, Значение функции потерь: {loss_value.item()}\r')

    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

KeyboardInterrupt: 